## Import Libraries

In [ ]:
import numpy as np
import os
import tarfile
import zipfile
import sys
import cv2
import csv
import time

import six.moves.urllib as urllib
import tensorflow as tf
from matplotlib import pyplot as plt
from collections import defaultdict

from io import StringIO
from PIL import Image

from utils import label_map_util
from utils import visualization_utils as vis_util

## Initialize .csv

In [ ]:
line_c = 'Vehicle Type/Size, Vehicle Color, Vehicle Movement Direction, Vehicle Speed (km/h)'
file1 = open('traffic.csv', 'w')
x = [line_c.split(',')]
try:
    writer = csv.writer(file1)
    writer.writerows(x)
finally:
    file1.close()

## "SSD with Mobilenet" model

In [ ]:
db = 'http://download.tensorflow.org/models/object_detection/'

nom = 'ssd_coco'
fom = nom + '.tar.gz'

## Path to frozen detection graph. This is the actual model that is used for the object detection.\

In [ ]:
ckpt = nom + '/fig.pb'

## List of the strings that is used to add correct label for each box.

In [ ]:
pl = os.path.join('data', 'lm_mscoco.pbtxt')

number_classes = 90

## Load a (frozen) Tensorflow model into memory

In [ ]:
graph_d = tf.Graph()
with graph_d.as_default():
    ogd = tf.GraphDef()
    
    file2 = tf.gfile.GFile(ckpt, 'rb')
    try:
        graph_s = file2.read()
        ogd.ParseFromString(graph_s)
        tf.import_graph_def(ogd, name='')
    finally:
        file2.close()

## Loading label map

In [ ]:
map_l = label_map_util.load_labelmap(pl)
cat = label_map_util.convert_label_map_to_categories(map_l, max_num_classes=number_classes, use_display_name=True)
cat_i = label_map_util.create_category_index(cat)

## Input Video

In [ ]:
video = cv2.VideoCapture('video.mp4')

# Detection

In [ ]:
def detect_object():
    t_vehicle = 0
    speed_of_vehicle = '..waiting..'
    direction_of_vehicle = '..waiting..'
    size_of_vehicle = '..waiting..'
    color_of_vehicle = '..waiting..'
    with graph_d.as_default():
        file3 = tf.Session(graph=graph_d)
        try:

            tensor_im = graph_d.get_tensor_by_name('image_tensor:0')

            box_d = graph_d.get_tensor_by_name('detection_boxes:0')

            score_d = graph_d.get_tensor_by_name('detection_scores:0')
            class_d = graph_d.get_tensor_by_name('detection_classes:0')
            num_d = graph_d.get_tensor_by_name('num_detections:0')

            while video.isOpened():
                (re, fr) = video.read()

                if not re:
                    print ('... end of video ...')
                    break

                fr_i = fr

                expand_im_np = np.expand_dims(fr_i, axis=0)

                (boxes, scores, classes, num) = \
                    file3.run([box_d, score_d,
                             class_d, num_d],
                             feed_dict={tensor_im: expand_im_np})

                (count, line_c) = \
                    vis_util.visualize_boxes_and_labels_on_image_array(
                    video.get(1),
                    fr_i,
                    np.squeeze(boxes),
                    np.squeeze(classes).astype(np.int32),
                    np.squeeze(scores),
                    cat_i,
                    use_normalized_coordinates=True,
                    line_thickness=4,
                    )

                t_vehicle = t_vehicle + count

                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(
                    fr_i,
                    'Detected Vehicles: ' + str(t_vehicle),
                    (10, 35),
                    font,
                    0.8,
                    (0, 0xFF, 0xFF),
                    2,
                    cv2.FONT_HERSHEY_SIMPLEX,
                    )

                if count == 1:
                    cv2.line(fr_i, (0, 200), (640, 200), (0, 0xFF, 0), 5)
                else:
                    cv2.line(fr_i, (0, 200), (640, 200), (0, 0, 0xFF), 5)

                cv2.rectangle(fr_i, (10, 275), (230, 337), (180, 132, 109), -1)
                cv2.putText(
                    fr_i,
                    'ROI Line',
                    (545, 190),
                    font,
                    0.6,
                    (0, 0, 0xFF),
                    2,
                    cv2.LINE_AA,
                    )
                cv2.putText(
                    fr_i,
                    'LAST PASSED VEHICLE INFO',
                    (11, 290),
                    font,
                    0.5,
                    (0xFF, 0xFF, 0xFF),
                    1,
                    cv2.FONT_HERSHEY_SIMPLEX,
                    )
                cv2.putText(
                    fr_i,
                    '-Movement Direction: ' + direction_of_vehicle,
                    (14, 302),
                    font,
                    0.4,
                    (0xFF, 0xFF, 0xFF),
                    1,
                    cv2.FONT_HERSHEY_COMPLEX_SMALL,
                    )
                cv2.putText(
                    fr_i,
                    '-Speed(km/h): ' + speed_of_vehicle,
                    (14, 312),
                    font,
                    0.4,
                    (0xFF, 0xFF, 0xFF),
                    1,
                    cv2.FONT_HERSHEY_COMPLEX_SMALL,
                    )
                cv2.putText(
                    fr_i,
                    '-Color: ' + color_of_vehicle,
                    (14, 322),
                    font,
                    0.4,
                    (0xFF, 0xFF, 0xFF),
                    1,
                    cv2.FONT_HERSHEY_COMPLEX_SMALL,
                    )
                cv2.putText(
                    fr_i,
                    '-Vehicle Size/Type: ' + size_of_vehicle,
                    (14, 332),
                    font,
                    0.4,
                    (0xFF, 0xFF, 0xFF),
                    1,
                    cv2.FONT_HERSHEY_COMPLEX_SMALL,
                    )

                cv2.imshow('vehicle detection', fr_i)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

                if line_c != 'not_available':
                    file4 = open('traffic.csv', 'a')
                    y = [line_c.split(',')]
                    try:
                        writer = csv.writer(file4)
                        (size_of_vehicle, color_of_vehicle, direction_of_vehicle, speed_of_vehicle) = \
                            line_c.split(',')
                        writer.writerows(y)
                    finally:
                        file4.close()
            video.release()
            cv2.destroyAllWindows()
            
        finally:
            file3.close()

In [ ]:
detect_object()